In [97]:
import os
from bs4 import BeautifulSoup
from time import sleep
from pymongo import MongoClient
from pprint import pprint
import concurrent.futures
import random
from copy import deepcopy

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [42]:
#helper files
import get_proxies as proxies
from extractors import get_data 

#### Pull listing_url documents from mongoDB

In [6]:
db = MongoClient().get_database('philadelphia_home_prices')

In [7]:
col = db.get_collection('listing_urls')

In [8]:
cursor = db.listing_urls.find({})
listings = list(cursor)

In [94]:
#convert dictionary values to list
listing_urls_list = []
for i in listings:
    url = i['url']
    listing_urls_list.append(url)

In [10]:
#check first 5 in list
listing_urls_list[0:5]

['https://www.realtor.com/realestateandhomes-detail/1232-N-Mascher-St-Apt-A_Philadelphia_PA_19122_M49182-95881',
 'https://www.realtor.com/realestateandhomes-detail/212-Carpenter-St-Rear-C_Philadelphia_PA_19147_M34206-29412',
 'https://www.realtor.com/realestateandhomes-detail/1211-E-Cheltenham-Ave_Philadelphia_PA_19124_M33700-20336',
 'https://www.realtor.com/realestateandhomes-detail/1002-Brandywine-St_Philadelphia_PA_19123_M39147-93479',
 'https://www.realtor.com/realestateandhomes-detail/2607-N-31st-St_Philadelphia_PA_19132_M32883-74202']

#### New collection for listing data

In [93]:
db.list_collection_names()

['prices_messy', 'listing_data', 'listing_urls']

In [12]:
#uncomment the code you need to use
#db.drop_collection('listing_data')
#db.create_collection('listing_data')

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'philadelphia_home_prices'), 'listing_data')

In [13]:
col = db.get_collection('listing_data')

In [14]:
col.count_documents({})

0

#### Get free proxies from https://free-proxy-list.net/

In [15]:
#call function to get list of proxies
proxies_address = proxies.proxy_list();

2020-11-04 17:11:11,831 root   DEBUG    === Initialized Proxy Parsers ===
2020-11-04 17:11:11,832 root   DEBUG    	 FreeProxy parser of 'http://free-proxy-list.net' with required bandwidth: '150' KBs
2020-11-04 17:11:11,833 root   DEBUG    	 PremProxy parser of 'https://premproxy.com/list/' with required bandwidth: '150' KBs
2020-11-04 17:11:11,833 root   DEBUG    =================================
2020-11-04 17:11:12,247 root   DEBUG    Added 300 proxies from FreeProxy
2020-11-04 17:11:12,809 http_request_randomizer.requests.parsers.PremProxyParser DEBUG    Pages: {'', '06.htm', '05.htm', '02.htm', '08.htm', '03.htm', '07.htm', '04.htm'}
2020-11-04 17:11:13,355 http_request_randomizer.requests.parsers.js.UnPacker INFO     JS UnPacker init path: https://premproxy.com/js/d9afb.js
2020-11-04 17:11:13,884 http_request_randomizer.requests.parsers.js.UnPacker DEBUG    portmap: {'r7974': '32108', 'r7d7a': '8080', 're822': '999', 'r90ea': '47464', 'r366e': '53281', 'rf7d9': '3128', 'rdd9a': '8

#### Pick which urls in list by index

In [16]:
listing_from = 0
listing_to = len(listing_urls_list)
broken_urls = []

In [99]:
#only run once
remaining_listing_urls_list = deepcopy(listing_urls_list)

In [100]:
def scrape(urls):
    listing_data = {}
    #while len(remaining_listing_urls_list) > =:
    try:
        PROXY = random.choice(proxies_address)
        webdriver.DesiredCapabilities.CHROME['proxy']={"httpProxy":PROXY,
                                                       "ftpProxy":PROXY,
                                                       "sslProxy":PROXY,
                                                       "proxyType":"MANUAL",}

        driver = webdriver.Chrome(chromedriver)
        url = random.choice(remaining_listing_urls_list)
        driver.get(url)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        listing_data = get_data(soup)

        #will skip and move url to bad_urls if the page format is different than expected
        if listing_data['Address'] == 'Not Found':
            bad_urls.append(url)
            remaining_listing_urls_list.remove(url)
        else:
            remaining_listing_urls_list.remove(url)
    
    except:
        print('except ran, bad proxy')
        proxies_address.remove(PROXY)
        print(len(proxies_address))
    
    col.insert_one(listing_data)
    
    driver.close()
    
    return listing_data

In [ ]:
# to us concurrent.furtures for multi-threading
def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):

In [104]:
count = 0
while count < 20:
    listing_data = scrape(remaining_listing_urls_list)
    #print(count)
    #print(listing_data)
    count += 1

2020-11-04 19:21:06,222 selenium.webdriver.remote.remote_connection DEBUG    POST http://127.0.0.1:52894/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "proxy": {"httpProxy": "200.35.56.161:35945", "ftpProxy": "200.35.56.161:35945", "sslProxy": "200.35.56.161:35945", "proxyType": "manual"}, "goog:chromeOptions": {"extensions": [], "args": []}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "proxy": {"httpProxy": "200.35.56.161:35945", "ftpProxy": "200.35.56.161:35945", "sslProxy": "200.35.56.161:35945", "proxyType": "MANUAL"}, "goog:chromeOptions": {"extensions": [], "args": []}}}
2020-11-04 19:21:07,279 selenium.webdriver.remote.remote_connection DEBUG    Finished Request
2020-11-04 19:21:07,281 selenium.webdriver.remote.remote_connection DEBUG    POST http://127.0.0.1:52894/session/64af112df5e64778f8a1d42efbd6dd69/url {"url": "https://www.realtor.com/realestateandhomes-detail/1600-Wal

https://www.realtor.com/realestateandhomes-detail/1600-Wallace-St_Philadelphia_PA_19130_M42439-74584


2020-11-04 19:21:37,327 selenium.webdriver.remote.remote_connection DEBUG    Finished Request
2020-11-04 19:21:37,327 selenium.webdriver.remote.remote_connection DEBUG    GET http://127.0.0.1:52894/session/64af112df5e64778f8a1d42efbd6dd69/source {}
2020-11-04 19:21:37,353 selenium.webdriver.remote.remote_connection DEBUG    Finished Request
2020-11-04 19:21:37,363 selenium.webdriver.remote.remote_connection DEBUG    DELETE http://127.0.0.1:52894/session/64af112df5e64778f8a1d42efbd6dd69/window {}
2020-11-04 19:21:37,425 selenium.webdriver.remote.remote_connection DEBUG    Finished Request


except ran, bad proxy
601
0
{}


2020-11-04 19:21:38,449 selenium.webdriver.remote.remote_connection DEBUG    POST http://127.0.0.1:52918/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "proxy": {"httpProxy": "85.238.167.170:51915", "ftpProxy": "85.238.167.170:51915", "sslProxy": "85.238.167.170:51915", "proxyType": "manual"}, "goog:chromeOptions": {"extensions": [], "args": []}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "proxy": {"httpProxy": "85.238.167.170:51915", "ftpProxy": "85.238.167.170:51915", "sslProxy": "85.238.167.170:51915", "proxyType": "MANUAL"}, "goog:chromeOptions": {"extensions": [], "args": []}}}
2020-11-04 19:21:39,417 selenium.webdriver.remote.remote_connection DEBUG    Finished Request
2020-11-04 19:21:39,425 selenium.webdriver.remote.remote_connection DEBUG    POST http://127.0.0.1:52918/session/9fce1392e8d617adf6bb4a49487c5ecc/url {"url": "https://www.realtor.com/realestateandhomes-detail/22

...

8
{'Address': '1002 Brandywine St,', 'Locality': 'Philadelphia', 'Region': 'PA', 'Postal Code': '19123', 'Price': ['320,000', '320,000', '320,000', '320,000', '320,000', '320,000', '320,000', '320,000'], 'Days on Market': ['43'], 'SQFT House': ['1,540', '1,540', '1,540', '1,540'], 'SQFT Lot': ['779', '779'], 'Full Baths': ['1'], 'Half Baths': [], 'Bedrooms': ['3'], 'Garage': [], 'Master Bath': [], 'Cooling': [' Window Unit(s)'], 'Heating': [' Forced Air Heating'], 'Annual Tax': ['4310'], 'House Type': [' End of Row/Townhouse'], 'Build Year': [' 1905', ' 1905'], 'Updated': [' 08/07/2020'], 'Neighborhood': ['NORTHERN LIBERTIES'], 'Neighborhood median sales': [], 'Neighborhood Median DOM': [], 'Neighborhood Price SQFT': ['239'], 'Association': ['No'], 'Association Monthly': ['0'], 'crawl time': datetime.datetime(2020, 11, 4, 19, 33, 11, 549286)}


2020-11-04 19:33:12,624 selenium.webdriver.remote.remote_connection DEBUG    POST http://127.0.0.1:53992/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "proxy": {"httpProxy": "82.137.255.109:8080", "ftpProxy": "82.137.255.109:8080", "sslProxy": "82.137.255.109:8080", "proxyType": "manual"}, "goog:chromeOptions": {"extensions": [], "args": []}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "proxy": {"httpProxy": "82.137.255.109:8080", "ftpProxy": "82.137.255.109:8080", "sslProxy": "82.137.255.109:8080", "proxyType": "MANUAL"}, "goog:chromeOptions": {"extensions": [], "args": []}}}
2020-11-04 19:33:13,889 selenium.webdriver.remote.remote_connection DEBUG    Finished Request
2020-11-04 19:33:13,890 selenium.webdriver.remote.remote_connection DEBUG    POST http://127.0.0.1:53992/session/f24599e0fb6b7e4b0d6e28eeffc934f9/url {"url": "https://www.realtor.com/realestateandhomes-detail/5731-N-1

https://www.realtor.com/realestateandhomes-detail/5731-N-17th-St_Philadelphia_PA_19141_M45402-65058


2020-11-04 19:33:43,994 selenium.webdriver.remote.remote_connection DEBUG    Finished Request
2020-11-04 19:33:43,994 selenium.webdriver.remote.remote_connection DEBUG    GET http://127.0.0.1:53992/session/f24599e0fb6b7e4b0d6e28eeffc934f9/source {}
2020-11-04 19:33:44,016 selenium.webdriver.remote.remote_connection DEBUG    Finished Request
2020-11-04 19:33:44,026 selenium.webdriver.remote.remote_connection DEBUG    DELETE http://127.0.0.1:53992/session/f24599e0fb6b7e4b0d6e28eeffc934f9/window {}
2020-11-04 19:33:44,080 selenium.webdriver.remote.remote_connection DEBUG    Finished Request


except ran, bad proxy
600
9
{}


In [106]:
"85.238.167.170:51915" in proxies_address

True

In [108]:
col.count_documents({})

0

In [111]:
'https://www.realtor.com/realestateandhomes-detail/4521-Tolbut-St_Philadelphia_PA_19136_M45152-02595' in remaining_listing_urls_list

False

In [113]:
len(remaining_listing_urls_list)

1086

In [114]:
len(listing_urls_list)

1100